## 2RPFS Problem (Cmax objective) - Data treatment of result files 

In [ ]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import os

%matplotlib inline

In [ ]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

### List files in the output folder 

In [ ]:
rootfolder = os.getcwd()
file_list = glob.glob(os.path.join(rootfolder, 'output') + '/*.csv', recursive=True)
file_list

### Read all the CSV files 

In [ ]:
# Alternative script to treat files with incorrect number of coluns or faulty lines
def alternative_csv_reader(filename, delimiter=',', header=0, names=None):
    with open(filename, 'r') as file:
        lines = file.readlines() 
        count = 1
        line_list = []
        num_columns = 20
        for line in lines:  # Strips the newline character 
            #print("line{}: {}".format(count, line.strip())) 
            nc = len(line.split(','))
            if 'executionId,' in line:
                #num_columns = nc
                print('Detected {0} columns in CSV file.'.format(nc))
            else:
                if 'none,' in line:
                    if nc == num_columns:
                        line_list.append(line)
                    elif nc > num_columns:  # treat strange truncated lines
                        line = line[line.rfind('none,'):]
                        nc = len(line.split(','))
                        if nc == num_columns:
                            print('WARN: truncating line {0}, for having more columns than expected.'.format(count))
                            line_list.append(line)
                        else:
                            print('WARN: Ignoring line {0}, since it has {1} columns, instead of {2}: '.format(count, nc, num_columns), line)
                    else:  # Ignore line
                        print('WARN: Ignoring line {0}: '.format(count), line)    
                elif len(line_list[-1].split(',')) < num_columns:  # current line is a continuation of the previous one
                    line_list[-1] = line_list[-1].replace('\n', '') + line
                    print('*** Treated line {0}: '.format(count), line_list[-1])
                else:  # Ignore line
                    print('WARN: Ignoring line {0}: '.format(count), line)
            count += 1
        # assert all lines have the same number of columns
        count = 1
        for line in line_list:
            nc = len(line.split(','))
            if nc != num_columns:
                print('ERROR: Line {0} has {1} columns, instead of {2}: '.format(count, nc, num_columns), line)
            count += 1
        text_data = StringIO(''.join(line_list))
        #print('line_list: ', str(line_list))
        #print('text_data: ', text_data)
        df = pd.read_csv(text_data, delimiter=delimiter, header=header, names=names)
        return df

### Process all CSV files and append all data to a single dataframe (one per solution method: Wilson, Wagner) 

In [ ]:
%%time

df_wilson = pd.DataFrame()
df_wagner = pd.DataFrame()
for filename in file_list:
    print('Processing file ', filename)
    try:
        df_ = pd.read_csv(filename, delimiter=',', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','budget_Gamma','cmax','permutation','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','cmax_dp'])
    except:  # try alternative method to read csv lines
        df_ = alternative_csv_reader(filename, delimiter=',', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','budget_Gamma','cmax','permutation','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','cmax_dp'])
    if 'wagner' not in filename:  # Wilson result file
        df_wilson = df_wilson.append(df_.copy())
    else:  # Wagner result file
        df_wagner = df_wagner.append(df_.copy())

In [ ]:
df_wilson

In [ ]:
df_wagner.info()

### Remove duplicated header rows from both dataframes 

In [ ]:
def find_invalid_values(df):
    all_invalid_values = set()
    for col in df:
        if col not in ['executionId','ub_name','instance_name','budget_Gamma','permutation','is_optimal','validated']:
            # 'alpha','n','m','cmax','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','gap','lb','cost','cmax_dp'
            a = pd.to_numeric(df[col], errors='coerce')
            idx = a.isna()
            invalid_values = df.loc[idx][col].unique()
            all_invalid_values.update(invalid_values)
        #elif col in ['is_optimal','validated']
    print('Invalid values:', all_invalid_values)
    return all_invalid_values

In [ ]:
find_invalid_values(df_wagner)

In [ ]:
def filter_invalid_values(df):
    # IMPORTANT: AVOID FILTERING 'NAN' VALUES
    for invalid_value in find_invalid_values(df):
        if isinstance(invalid_value, str):  # Evita filtrar os nan
            df = df[~(df == invalid_value).any(axis=1)]
    return df

In [ ]:
df_wilson = filter_invalid_values(df_wilson)
find_invalid_values(df_wilson)

In [ ]:
df_wagner = filter_invalid_values(df_wagner)
find_invalid_values(df_wagner)

### Convert column types from object 

In [ ]:
def convert_column_types(df):
    for col in df:
        if col in ['alpha','n','m','cmax','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','gap','lb','cost','cmax_dp']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif col in ['is_optimal','validated']:
            df[col] = df[col].astype('bool')
    return df

In [ ]:
%%time
df_wilson = convert_column_types(df_wilson)
df_wagner = convert_column_types(df_wagner)
df_wagner.dtypes

### Trim existing string columns 

In [ ]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

In [ ]:
df_wilson = trim_all_columns(df_wilson)
df_wagner = trim_all_columns(df_wagner)

### Include a column with the name of the underlying C&CG MILP Model

In [ ]:
df_wilson['model'] = 'Wilson'
df_wagner['model'] = 'Wagner'

### Concatenate dataframes 

In [ ]:
df = df_wilson.append(df_wagner)
df.head(4)

### Split the column budget_Gamma into Gamma1 and Gamma2 

In [ ]:
# new data frame with split value columns 
new = df["budget_Gamma"].str.split(" ", n = 1, expand = True) 
# making separate first name column from new data frame 
df["Gamma1"]= new[0] 
# making separate last name column from new data frame 
df["Gamma2"]= new[1] 
# convert Gamma columns to numeric
df["Gamma1"] = pd.to_numeric(df["Gamma1"], errors='coerce')
df["Gamma2"] = pd.to_numeric(df["Gamma2"], errors='coerce')

### Fix instance names 

The original instance names, as provided by Ying in the instance file zip, were assembled incorrectly.

The problem lies in the alpha percentage. We are now going to fix this issue.

In [ ]:
df['seq'] = df['instance_name'].str[7:9]
df['n_str'] = df['n'].astype(str).str.zfill(3)
df['alpha_str'] = df['alpha'].astype(str)

In [ ]:
df['instance_name'] = 'RB' + df['n_str'] + df['alpha_str'] + df['seq'] + '.txt'
df.drop(columns=['n_str', 'alpha_str'], inplace=True)

### Round columns containing time (in seconds) 

In [ ]:
df['time_spent'] = df['time_spent'].round(2)
df['time_to_best_sol'] = df['time_to_best_sol'].round(2)

### Sort data according to model, instance_name, alpha, n, m, Gamma1 and Gamma2 and set index

In [ ]:
print('Sorting dataset...')
df = df.sort_values(['model', 'n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2'])
display(df.dtypes)
df = df.set_index(['model', 'n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2'])
display(df.head(6))

### Find missing results, for a given value of alpha, n and m

For a given group of alpha, n, m and budget_Gamma, there should be 10 results.

First we will build a dataframe with the instances list and all required budget values.

In [ ]:
data = []
rootfolder = os.getcwd()
jobs_folders = glob.glob(os.path.join(rootfolder, 'instances', 'robust', 'ying', 'data', '*/'), recursive=False)
for job_path in jobs_folders:
    alpha_folders = glob.glob(os.path.join(job_path, '*/'), recursive=False)
    n = job_path[job_path.find('data')+5:job_path.rfind('jobs')]
    #print('n: {}'.format(n))
    for alpha_path in alpha_folders:
        alpha = alpha_path[alpha_path.find('jobs')+5:alpha_path.rfind('%')]
        #print('alpha: {}'.format(alpha))
        instance_paths = glob.glob(os.path.join(alpha_path, '*'), recursive=False)
        for instance_path in instance_paths:
            instance_name = instance_path[instance_path.find('%')+2:]
            #print(instance_name)
            for gamma1 in [20, 40, 60, 80, 100]:
                for gamma2 in [20, 40, 60, 80, 100]:
                    for model in ['Wilson', 'Wagner']:
                        data.append([model, instance_name, alpha, n, 2, gamma1, gamma2])
df_instances = pd.DataFrame(data, columns=['model', 'instance_name', 'alpha', 'n', 'm', 'Gamma1', 'Gamma2'])
for col in df_instances:
    if col in ['alpha','n','m','Gamma1','Gamma2']:
        df_instances[col] = pd.to_numeric(df_instances[col], errors='coerce')
display(df_instances.dtypes)
df_instances = df_instances.set_index(['model', 'n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2'])
display(df_instances)

Now, lets join the instances dataframe with the results one (left join).

In [ ]:
df_joined = df_instances.join(df, how='left')
df_joined

Now we will export to CSV a list with all rows with NaN values (missing experimental results).

In [ ]:
missing_df = df_joined[df_joined.isnull().any(axis=1)].reset_index()[['model', 'n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2']]
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
if not os.path.exists(outputfolder):
    os.makedirs(outputfolder)
print('Saving file on folder: ' + outputfolder)
fname = os.path.join(outputfolder, '2RPFS_Cmax_missing_results.csv')
missing_df.to_csv(fname, sep=';')
print('Saved: ' + fname)

In [ ]:
df_grouped = df.groupby(['alpha', 'n', 'm', 'budget_Gamma']).agg({'executionId' : ['count']}).reset_index()
df_grouped.columns = [ ' '.join(str(i) for i in col) for col in df_grouped.columns]
#df_grouped.reset_index(inplace=True)
df_grouped

In [ ]:
table = pd.pivot_table(df, values='executionId', index=['alpha', 'n'], columns=['Gamma1', 'Gamma2'], aggfunc='count', fill_value=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(table)

### Export the dataset to CSV file 

In [ ]:
%%time

print('Saving file on folder: ' + outputfolder)
fname = os.path.join(outputfolder, '2RPFS_Cmax_all_results.csv')
df.to_csv(fname, sep=';')
print('Saved: ' + fname)